In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import transbigdata as tbd
import warnings
from pyproj import CRS

import pyvista as pv
from scripts import bd_preprocess
from scripts import footprint_insolation
warnings.filterwarnings('ignore')

# 网格分辨率(米)
resolution = 10

# 读取建筑物数据
building = gpd.read_file('data/苏州建筑.geojson')
building.crs = {'init': 'epsg:4326'}

# 预处理建筑物数据
building = bd_preprocess.bd_preprocess(building)

# 转为投影坐标系
lon = building['geometry'].iloc[0].centroid.x
lat = building['geometry'].iloc[0].centroid.y
project_epsg = CRS.from_proj4(
    "+proj=aeqd +lat_0="+str(lat)+" +lon_0="+str(lon)+" +datum=WGS84")
building = building.to_crs(project_epsg)

# 简化建筑，避免同位置的点出现
building['geometry'] = building.simplify(0.001)
building['roof_area'] = building.area

In [2]:
# 生成建筑物的三角网格
building_mesh, tri_info = footprint_insolation.generate_mesh(building)

'''
输出结果中，building_mesh为建筑物的三角网格，tri_info为三角网格的信息

tri_info中的信息包括：
- index_tri: 三角网格的id
- building_id: 三角网格所属的建筑物的id
- type: 标识这一三角网格是该建筑的roof还是facade
- facade_id: 如果这一三角网格是墙面，则记录其所属的墙面的id
- entit_area: 这一三角网格所属的 屋顶 或 墙面 的整体面积，并非三角网格自身的面积，单位为平方米
- area: 该三角网格的面积，单位为平方米
'''


Generating 3D mesh: 100%|██████████| 28/28 [00:00<00:00, 252.20it/s]


'\n输出结果中，building_mesh为建筑物的三角网格，tri_info为三角网格的信息\n\ntri_info中的信息包括：\n- index_tri: 三角网格的id\n- building_id: 三角网格所属的建筑物的id\n- type: 标识这一三角网格是该建筑的roof还是facade\n- facade_id: 如果这一三角网格是墙面，则记录其所属的墙面的id\n- entit_area: 这一三角网格所属的 屋顶 或 墙面 的整体面积，并非三角网格自身的面积，单位为平方米\n- area: 该三角网格的面积，单位为平方米\n'

In [3]:
bd_tri = tri_info.fillna(0).groupby(['building_id','entity_area','type','facade_id'])['area'].sum().reset_index()

In [4]:
bd_tri

,building_id,entity_area,type,facade_id,area
0,0,21.557043,facade,3.0,21.557043
1,0,22.202204,facade,1.0,22.202204
2,0,31.229381,facade,2.0,31.229381
3,0,34.238777,facade,0.0,34.238777
4,0,79.363699,roof,0.0,79.363699
...,...,...,...,...,...
178,27,908.950657,facade,2.0,908.950657
179,27,1017.761119,roof,0.0,1017.761119
180,27,1272.609065,facade,0.0,1272.609065
181,27,1355.297462,facade,7.0,1355.297462


In [4]:
# 计算每个三角网格的日照时数
'''
其中，mesh为三角网格的pyvista对象，已经根据日照时长着色

tri_hours为每个三角网格的id和日照时长，两列，第一列为三角网格的id，第二列为日照时长
'''
mesh,tri_solar,tri_hours = footprint_insolation.solar_insolation(
    building_mesh,lon,lat,
    dates = [str(pd.to_datetime('2022-01-01') + pd.to_timedelta(i, unit='day'))[:10] for i in range(1)],
    time_precision=3600,padding=1800,method=1)


In [5]:
#tri_solar为每个三角网格的日照情况，两列，第一列为三角网格的id，第二列为被日照的时间点
tri_solar

,index_tri,date
0,1111399,2022-01-01 07:53:02.271427072
2,1111380,2022-01-01 07:53:02.271427072
3,321920,2022-01-01 07:53:02.271427072
4,342314,2022-01-01 07:53:02.271427072
8,930245,2022-01-01 07:53:02.271427072
...,...,...
11454302,1265183,2022-01-01 15:53:02.271427072
11454382,1264615,2022-01-01 15:53:02.271427072
11454396,1264577,2022-01-01 15:53:02.271427072
11454464,1264949,2022-01-01 15:53:02.271427072


In [6]:
#tri_hours为每个三角网格的id、日照时长
tri_hours

,index_tri,Hours
0,117,1.0
1,125,1.0
2,127,1.0
3,129,1.0
4,134,1.0
...,...,...
409298,1269840,9.0
409299,1269842,9.0
409300,1269844,9.0
409301,1269854,9.0


In [7]:
# 保存mesh结果用于可视化。如果报错，则将draco_export_settings选项去掉。
_ = mesh.export(f'Valencia_Spain.glb',
                draco_export_settings={"gltf_draco_mesh_compression_level": 6,  # draco压缩设置
                                       "gltf_draco_position_quantization": 14,
                                       "gltf_draco_normal_quantization": 10,
                                       "gltf_draco_texcoord_quantization": 12,
                                       "gltf_draco_color_quantization": 10,
                                       "gltf_draco_generic_quantization": 12})

DracoEncoder | Preserve triangle order: no
DracoEncoder | Encoded 3818334 vertices, 3818334 indices, raw size: 76366680, encoded size: 10562012, compression ratio: 7.23
